In [108]:
%matplotlib qt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from ast import literal_eval
from pprint import pprint
import pandas as pd
import time
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [2]:
driver = webdriver.Firefox()
driver.get(r'https://www.dubicars.com/dubai/used/mercedes-benz/c-class?o=')


In [64]:
total_listing = []
while True:
    try:
        page = bs(driver.page_source)
        for items in page.find('section', {'id':'serp-list-new'}).find_all('li'):
    #         print(items.prettify())
            try:
                total_listing.append({
                    'item_id': items['data-item-id'],
                    'km': items['data-item-kilometers'],
                    'price': items['data-item-price'],
                    'title': items['data-item-title'],
                    'year': items.find_all('span',{'class':'info'})[0].text,
                    'link': items.a['href'],
                })
            except Exception as e:
                print('Error:',e)
                continue
    #         break
        driver.find_element_by_id('pagination-desktop').find_element_by_class_name('next').click()
    except Exception as e:
        print(e)
        print('end of scrapping')
        break

Error: 'data-item-id'
Error: 'data-item-id'
Error: 'data-item-id'
Error: 'data-item-id'
Message: Unable to locate element: .next

end of scrapping


In [117]:
result = pd.DataFrame(total_listing)
result['km'] = result.km.map(lambda x: x.replace(' km', '')).map(lambda x: x.replace(',',''))
result['price'] = pd.to_numeric(result.price).fillna(0)
result['km'] = pd.to_numeric(result.km).fillna(0)
result['year'] = pd.to_numeric(result.year).fillna(0)
result.head()

,item_id,km,link,price,title,year
0,248174,11000,https://www.dubicars.com/2017-mercedes-benz-c-...,349000.0,"Mercedes-Benz C 63 AMG s, COUPE,EDITION ONE,GC...",2017
1,248711,48800,https://www.dubicars.com/2016-mercedes-benz-c-...,122000.0,Mercedes-Benz C 200,2016
2,239073,18200,https://www.dubicars.com/2013-mercedes-benz-c-...,279000.0,Mercedes-Benz C 63 Coupe Black Series (Track P...,2013
3,242379,53000,https://www.dubicars.com/2013-mercedes-benz-c-...,65000.0,Mercedes-Benz C 200,2013
4,245092,8000,https://www.dubicars.com/2017-mercedes-benz-c-...,270000.0,Mercedes-Benz C 43 AMG,2017


In [102]:
result[['title','km']].groupby('title',as_index=False).count().sort_values(by='km',ascending=False)

,title,km
2,Mercedes-Benz C 200,18
19,Mercedes-Benz C 300,10
14,Mercedes-Benz C 250,8
22,Mercedes-Benz C 300 Coupe,6
13,Mercedes-Benz C 230,4
39,Mercedes-Benz C 63 Coupe,4
29,Mercedes-Benz C 63 AMG,4
26,Mercedes-Benz C 43 AMG,4
0,Mercedes-Benz C 180,3
8,Mercedes-Benz C 200 G.C.C FULLY LOADED FULL //...,2


In [128]:
result[result.price != 0].groupby('year').agg({'price':['min','mean','max']}).plot(kind='bar')

In [129]:
result[result.km != 0].groupby('year').agg({'km':['min','mean','max']}).plot(kind='bar')